# Clean data and update company collection

In [1]:
from pymongo import MongoClient
import pandas as pd
import functions as F

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [3]:
projection = {"name":1,"offices":1,"category_code":1, "founded_year":1}
offices = list(db.companies.find({"$and":[{"deadpooled_year":None},
                                        {"founded_year":{"$gte":2009}}]},projection))


In [4]:
df = pd.DataFrame(offices)

In [5]:
cleanData = F.CleanDataFrame(df,"offices")
cleanData = cleanData.drop(columns=["_id","offices"])
cleanData["location"] = cleanData[["latitude"
                                   ,"longitude"]].apply(lambda x:F.asGeoJSON(x.longitude,x.latitude), axis=1)
cleanData[["latitude","longitude","location"]]
cleanData = cleanData.drop(columns=["latitude","longitude"])

In [11]:
cleanData.to_json("../OUTPUT/cleaned_companies.json", orient="records")
# mongoimport --db companies --collection clean_companies --OUTPUT/cleaned_companies --jsonArray

In [34]:
offices = list(db.cleaned_companies.find({}))
count = 0
for i in offices:
    if type(i['location']['coordinates'][0])==type(None):
        db.cleaned_companies.delete_one(i)